sources:
docs.streamlit.io/library/get-started

# Introduction

In [1]:
# Import Libraries
import numpy as np
import pandas as pd

pd.set_option("display.precision", 2)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
from pandas.api.types import (
    is_categorical_dtype,
    is_datetime64_any_dtype,
    is_numeric_dtype,
    is_object_dtype,
)

import plotly.figure_factory as ff

## Streamlit

In [2]:
import streamlit as st

In [3]:
st.set_page_config(
    page_title="9base",
    page_icon="⚾",
    layout="wide",
    initial_sidebar_state="expanded",
    menu_items={
        'ninebase Info': 'https://www.extremelycoolapp.com/help',
        'Suggest a Feature!': "https://www.extremelycoolapp.com/bug",
        'About': "# This is a header. This is an *extremely* cool app!"
    }
)

StreamlitAPIException: We only accept the keys: "Get help", "Report a bug", and "About" ("ninebase info" is not a valid key.)

In [ ]:
#from st_pages import Page, Section, show_pages, add_page_title

##⚾🥇

#add_page_title() # By default this also adds indentation

## Specify what pages should be shown in the sidebar, and what their titles and icons should be
#show_pages(
#    [
#        Section("14 Day", icon="🥇"),
#        Section("Season", icon="♂️"),
#        Page("player_batting_season.py", "Player Batting", "🏠"),
#    ]
#)

## Load Data

In [4]:
# Format for GitHub
df = pd.read_csv('data_storage/player_batting_season.csv')

# Format for Jupyter Notebook
#df = pd.read_csv('C:/Users/b7tbu/NINEBASE/ninebase/data_storage/player_batting_season.csv')

In [ ]:
df.head()

## Scatter Chart

In [ ]:
import plotly.express as px

In [8]:
fig = px.scatter(
    df.query("Season==2023"),
    x = "wRC+",
    y = "Hard%+",
    color = "Team",
    hover_name = "Name",
    log_x = True,
    size_max = 60,
)

In [9]:
#st.plotly_chart(fig, theme="streamlit", use_container_width = True)

## Filters

In [10]:
# calculate min/max/mean for slider
min_AB = df['AB'].min()
max_AB = df['AB'].max()
mean_AB = df['AB'].mean()

# float em
min_AB = float(min_AB)
max_AB = float(max_AB)
mean_AB = float(mean_AB)

In [11]:
# slider

ab_slider = st.slider('At Bat Range', min_AB, max_AB, (mean_AB, max_AB))


2023-07-08 09:27:04.206 
  command:

    streamlit run C:\Users\b7tbu\anaconda3\envs\ninebase\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [12]:
## Sidebar

## Columns

In [13]:
col1, col2 = st.columns([2,1])

In [14]:
col1.subheader("Scatter Chart")
col1.plotly_chart(fig, theme="streamlit", use_container_width = True)

col2.subheader("Filters")
col2.write(ab_slider)

## Raw Data Display

In [9]:
from st_aggrid import AgGrid, GridOptionsBuilder, DataReturnMode, JsCode

In [10]:
# Simplify dataframe, narrow to most insightful columns
df_short = df[['Name', 'Team', 'Age', 'AB', 'BB%+', 'K%+', 'BABIP+', 'Hard%+', 'wRC+']]

In [25]:
#builds a gridOptions dictionary using a GridOptionsBuilder instance.
builder = GridOptionsBuilder.from_dataframe(df_short)
builder.configure_column("Name", header_name="First", editable=False)
go = builder.build()

In [26]:
#uses the gridOptions dictionary to configure AgGrid behavior.
AgGrid(df_short, gridOptions=go)

2023-07-08 13:49:00.009 
  command:

    streamlit run C:\Users\b7tbu\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


AgGridReturn(data=                      Name Team  Age   AB  BB%+  K%+  BABIP+  Hard%+  wRC+
0         Ronald Acuna Jr.  ATL   25  347   122   55     114     135   168
1            Shohei Ohtani  LAA   28  334   144   97     105     124   177
2             Mookie Betts  LAD   30  325   152   74      89     134   150
3            Wander Franco  TBR   22  331    95   59     103     109   125
4           Corbin Carroll  ARI   22  297   101   88     106     100   145
5              Luis Robert  CHW   25  327    69  123     109     106   144
6          Freddie Freeman  LAD   33  347   113   76     119     121   152
7                Juan Soto  SDP   24  298   238   89     105     117   153
8       Fernando Tatis Jr.  SDP   24  282    90   85     104     124   139
9             Jose Ramirez  CLE   30  325   128   42      97      93   132
10           Adolis Garcia  TEX   30  336   110  111     100     140   131
11              Mike Trout  LAA   31  304   150  126     114     125   137
12     

ValueError: GridUpdateMode should be either a valid GridUpdateMode enum value or string

grid_table = AgGrid(df_short,
                   gridOptions=gridOptions,
                   fit_columns_on_grid_load=True,
                   height='400',
                   width='100%',
                   theme="streamlit",
                   update_mode=GridUpdateMode.GRID_CHANGED,
                   reload_data=True,
                   allow_unsafe_jscode=True,
                   editable=False
                   )

from st_aggrid import AgGrid, GridOptionsBuilder
import streamlit as st

from st_aggrid import agstyler
from src.agstyler import PINLEFT, PRECISION_ZERO, PRECISION_ONE, PRECISION_TWO, draw_grid

formatter = {
    'Name': ('Player', PINLEFT),
    'Team': ('Team', {'width': 80}),
    'Age': ('Age', {'width': 80}),
    'AB': ('At Bats', {'width': 80}),
    'K%+': ('K%+', {**PRECISION_ZERO,'width': 80}),
    'BABIP+': ('BABIP+', {**PRECISION_ZERO,'width': 80}),
    'Hard%+': ('Hard%+', {**PRECISION_ZERO,'width': 80}),
    'wRC+': ('wRC+', {**PRECISION_ZERO,'width': 80}),
}

row_number = st.number_input('Number of rows', min_value=0, value=20)
data = draw_grid(
    df.head(15),
    formatter=formatter,
    fit_columns=True,
    selection='multiple',  # or 'single', or None
    use_checkbox='True',  # or False by default
    max_height=300
)